# calculate rotation and transposition matrix

In [1]:
import os

data_folder = r'D:\Shiwei\20210624-P_Forebrain_smFISH_CTP09'
before_position_file = os.path.join(data_folder, '10x_positions_before.txt')
after_position_file = os.path.join(data_folder, '10x_positions_after.txt')


In [2]:
import numpy as np
import os, sys
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [3]:
R, T = align_manual_points(before_position_file, after_position_file, save=False)


In [4]:
R, T

(array([[ 0.99998462, -0.00554629],
        [ 0.00554629,  0.99998462]]),
 array([ 521.80790523, -404.95187017]))

# transpose 60x positions

In [5]:
old_positions = np.loadtxt(os.path.join(data_folder, 'positions_all.txt'), delimiter=',')

In [6]:
old_positions

array([[-2035.4,  5693.3],
       [-2035.4,  5903.3],
       [-2035.4,  6113.3],
       [-2035.4,  6323.3],
       [-2035.4,  6533.3],
       [-1825.4,  6533.3],
       [-1825.4,  6323.3],
       [-1825.4,  6113.3],
       [-1825.4,  5903.3],
       [-1825.4,  5693.3],
       [-1615.4,  5693.3],
       [-1615.4,  5903.3],
       [-1615.4,  6113.3],
       [-1615.4,  6323.3],
       [-1615.4,  6533.3],
       [-1405.4,  6533.3],
       [-1405.4,  6323.3],
       [-1405.4,  6113.3],
       [-1405.4,  5903.3],
       [-1405.4,  5693.3],
       [-1195.4,  5693.3],
       [-1195.4,  5903.3],
       [-1195.4,  6113.3],
       [-1195.4,  6323.3],
       [-1195.4,  6533.3],
       [ -985.4,  6533.3],
       [ -985.4,  6323.3],
       [ -985.4,  6113.3],
       [ -985.4,  5903.3],
       [ -985.4,  5693.3],
       [  575.6,  5488.8],
       [  575.6,  5698.8],
       [  575.6,  5908.8],
       [  575.6,  6118.8],
       [  575.6,  6328.8],
       [  785.6,  6328.8],
       [  785.6,  6118.8],
 

In [7]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[-1481.98408432  5299.54948513]
 [-1480.819363    5509.54625517]
 [-1479.65464167  5719.5430252 ]
 [-1478.48992035  5929.53979523]
 [-1477.32519902  6139.53656527]
 [-1267.32842899  6138.37184394]
 [-1268.49315032  5928.37507391]
 [-1269.65787164  5718.37830387]
 [-1270.82259297  5508.38153384]
 [-1271.98731429  5298.38476381]
 [-1061.99054426  5297.22004248]
 [-1060.82582293  5507.21681252]
 [-1059.66110161  5717.21358255]
 [-1058.49638028  5927.21035258]
 [-1057.33165896  6137.20712262]
 [ -847.33488893  6136.04240129]
 [ -848.49961025  5926.04563126]
 [ -849.66433158  5716.04886122]
 [ -850.8290529   5506.05209119]
 [ -851.99377423  5296.05532116]
 [ -641.99700419  5294.89059983]
 [ -640.83228287  5504.88736987]
 [ -639.66756154  5714.8841399 ]
 [ -638.50284022  5924.88090993]
 [ -637.33811889  6134.87767997]
 [ -427.34134886  6133.71295864]
 [ -428.50607018  5923.71618861]
 [ -429.67079151  5713.71941858]
 [ -430.83551283  5503.72264854]
 [ -432.00023416  5293.72587851]
 [ 1127.84

In [8]:
save_filename = os.path.join(data_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Shiwei\20210624-P_Forebrain_smFISH_CTP09\translated_positions_all.txt


# further adjust manually

In [9]:
adjust_before = np.array([[1474.18, 4239.34],[-1481.98, 5299.55],[1834.34, 2895.82]])

adjust_after = np.array([[1323.5, 4210.2],[-1630.9, 5270.5],[1683.6, 2866.5]])


delta_adjust = adjust_after - adjust_before
delta_adjust

array([[-150.68,  -29.14],
       [-148.92,  -29.05],
       [-150.74,  -29.32]])

In [10]:
manual_shift= np.mean(delta_adjust,axis=0)
manual_shift

array([-150.11333333,  -29.17      ])

In [11]:
#manual_shift = np.array([-28.1,  -8.7])
adjusted_new_positions = new_positions + manual_shift

In [12]:
adj_save_filename = os.path.join(data_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

D:\Shiwei\20210624-P_Forebrain_smFISH_CTP09\adjusted_translated_positions_all.txt
